# Simple ML Jump (STUDENT VERSION)
Billy Hau - June 29, 2022

- Data Cleaning / Feature Engineering / Modeling 
- Deployment

In [218]:
# Import Libraries
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import multiprocessing as mp

In [219]:
# Read CSV File
df = pd.read_csv("./data.csv")
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,8.601720,14.010661,16.942482,14.598472,False,True,False,False
1,5.087775,14.010661,13.281113,14.598472,False,True,False,False
2,1.361419,14.010661,9.398420,14.598472,False,False,False,False
3,6.708690,14.598472,-1.000000,-1.000000,False,False,False,True
4,5.509425,14.598472,-1.000000,-1.000000,True,False,False,False
...,...,...,...,...,...,...,...,...
65829,8.943563,12.793562,16.841225,15.542326,False,True,False,False
65830,5.539239,12.793562,12.705462,15.542326,False,True,False,False
65831,2.130824,12.793562,8.564732,15.542326,False,True,False,False
65832,4.672658,15.542326,-1.000000,-1.000000,False,True,True,False


In [220]:
# What do each of the columns mean?

In [221]:
jump = df[df['Jump'] == True]
dead = jump['Dead']
# dead.value_counts()
for i in range(dead.shape[0]):
    # print(jump.index[i])
    try:
        if any(any(df['Dead'][jump.index[i + y] + x] for x in range(0, 11)) for y in range(0, 2)) or not any(df['Scored'][jump.index[i] + x] for x in range(0, 5)):
            df.drop(df[df.index == jump.index[i]].index, inplace=True)
    except:
        pass
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,8.601720,14.010661,16.942482,14.598472,False,True,False,False
1,5.087775,14.010661,13.281113,14.598472,False,True,False,False
2,1.361419,14.010661,9.398420,14.598472,False,False,False,False
3,6.708690,14.598472,-1.000000,-1.000000,False,False,False,True
5,1.846144,14.598472,-1.000000,-1.000000,False,False,False,False
...,...,...,...,...,...,...,...,...
65829,8.943563,12.793562,16.841225,15.542326,False,True,False,False
65830,5.539239,12.793562,12.705462,15.542326,False,True,False,False
65831,2.130824,12.793562,8.564732,15.542326,False,True,False,False
65832,4.672658,15.542326,-1.000000,-1.000000,False,True,True,False


In [222]:
# dead.value_counts()
for i in range(df.shape[0]):
    # print(jump.index[i])
    try:
        if any(df['Dead'][df.index[i] + x] for x in range(0, 6)):
            df.drop(df[df.index == df.index[i]].index, inplace=True)
    except:
        pass
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,8.601720,14.010661,16.942482,14.598472,False,True,False,False
1,5.087775,14.010661,13.281113,14.598472,False,True,False,False
2,1.361419,14.010661,9.398420,14.598472,False,False,False,False
3,6.708690,14.598472,-1.000000,-1.000000,False,False,False,True
6,-1.000000,-1.000000,-1.000000,-1.000000,False,True,True,False
...,...,...,...,...,...,...,...,...
65825,22.173317,12.793562,32.913460,15.542326,False,True,False,False
65826,18.758329,12.793562,28.764744,15.542326,False,True,False,False
65828,12.151989,12.793562,20.738997,15.542326,False,True,False,False
65830,5.539239,12.793562,12.705462,15.542326,False,True,False,False


In [223]:
# Are there data we don't care about?
df.drop(df[df['Dead'] == True].index, inplace=True)
df = df.drop(["Dead", "Scored"], inplace=False, axis=1)
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded
0,8.601720,14.010661,16.942482,14.598472,False,True
1,5.087775,14.010661,13.281113,14.598472,False,True
2,1.361419,14.010661,9.398420,14.598472,False,False
3,6.708690,14.598472,-1.000000,-1.000000,False,False
7,39.553391,14.685490,54.367828,15.799332,False,True
...,...,...,...,...,...,...
65824,25.587893,12.793562,37.061676,15.542326,False,True
65825,22.173317,12.793562,32.913460,15.542326,False,True
65826,18.758329,12.793562,28.764744,15.542326,False,True
65828,12.151989,12.793562,20.738997,15.542326,False,True


In [224]:
jumpCount = df[df['Jump'] == True].shape[0]
jumpCount

217

In [225]:
noJumpCount = df[df['Jump'] == False].shape[0]
noJumpCount

59713

In [226]:
df_toDrop = df[df['Jump'] == False].sample(noJumpCount - jumpCount)
df.drop(df_toDrop.index, inplace=True)

In [227]:
# Create the Input DataFrame
X = df[['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed']]
X = X.to_numpy()

In [228]:
# Create the Output DataFrame and Map True / False to 1 / 0
Y = df[['Jump']]


def bool2Num(row):
    if row['Jump']:
        return 1
    else:
        return 0


Y['Jump'] = Y.apply(bool2Num, axis='columns')
Y = Y['Jump'].to_numpy()

/tmp/ipykernel_549164/2096279593.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Jump'] = Y.apply(bool2Num, axis='columns')


In [229]:
# Train Test Split with test size set to 20%
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [230]:
from sklearn.linear_model import LogisticRegression

# Define and Build Model (Logistic Regression)
model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [231]:
predict = model.predict(x_test)

In [232]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [233]:
# Print Out Prediction Accuraccy
accuracy_score(y_test, predict)

0.8888888888888888

In [234]:
# Try a Different Model (Random Forest)
"""model2 = ________________________________
model2.__________________________

predict = ___________________________
accuracy_score(_________________, ________________)"""


'model2 = ________________________________\nmodel2.__________________________\n\npredict = ___________________________\naccuracy_score(_________________, ________________)'

# Deployment

In [235]:
# Command to Open SIMPLE ML Jump 2 (for Example)

# Windows OS
#env_path = "D:\\User\\Desktop\\10botics Data Science\\ML Game\\SimpleMLJump2 Builds\\Windows\\Simple ML Jump 2.exe"

# MacOS
#env_path = 'open -n "/Users/billwaa/Desktop/ML Game/Simple ML Jump 2.app"' 

env_path = "../Linux/SimpleMLJump2.x86_64"

In [236]:
# Import Libraries
import socket
import struct
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import time
import subprocess
import platform
import collect
import multiprocessing as mp


# Find Open Socket Ports
def findOpenSockets(size: int):
    sock = []
    port = []

    for i in range(size):
        sock.append(socket.socket())
        sock[i].bind(('localhost', 0))
        port.append(sock[i].getsockname()[1])

    for i in range(len(sock)):
        sock[i].close()

    return port


# Find Open Ports and Launch Game Environment
envNum = 1
openPorts = findOpenSockets(envNum * 2)
gamePort = openPorts[0]
apiPort = openPorts[1]

# Determine OS and Launch 
if platform.system() == 'Darwin':
    env_path += f" --args --apiPort {str(apiPort)} --gamePort {str(gamePort)} --small false"
    process = subprocess.Popen([env_path], shell=True)
else:
    process = subprocess.Popen([env_path, '--apiPort', str(apiPort), '--gamePort', str(gamePort), '--small', 'false'])

# Establish UDP Network Client
localIP = "localhost"
bufferSize = 1024
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPServerSocket.bind((localIP, apiPort))

# Set UDP Timeout
UDPServerSocket.settimeout(3)

# Wait for Game Environment to Open
t0 = time.time()

while (time.time() - t0 < 5):
    pass


# Extract Data
def extractData(data):
    dist1 = struct.unpack('f', data[:4])
    speed1 = struct.unpack('f', data[4:8])
    dist2 = struct.unpack('f', data[8:12])
    speed2 = struct.unpack('f', data[12:16])

    onGround = True if (data[16] & (1 << 0)) == 1 else False
    toJump = True if (data[17] & (1 << 0)) == 1 else False
    isDead = True if (data[18] & (1 << 0)) == 1 else False
    scored = True if (data[19] & (1 << 0)) == 1 else False

    dat = [dist1[0], speed1[0], dist2[0], speed2[0], toJump, onGround, isDead,
           scored]  # Swap Order to Accomadate Old Model
    df_temp = pd.DataFrame(dat).transpose()

    return df_temp


# Reset Simulation
UDPServerSocket.sendto(bytes.fromhex('07 00'), (localIP, gamePort))

2

In [237]:
from os.path import exists

df2 = pd.DataFrame()
prevJump = False
# Process Loop - Retrieve Data from Simulation, Run Through Model, Output Command
while True:
    try:
        bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
        t0 = time.time()
        message = bytesAddressPair[0]
        address = bytesAddressPair[1]
        df_temp = extractData(message)
        X = df_temp.iloc[:, :4]
        X = X.astype(float)
    # Data PreProcessing Function Go Here
    # Model Prediction Function Go Here
        predict = model.predict(X)
        print(predict, end='\r')
    # If Jump Decision is Made, Send to Game
        if predict[0] > 0.5 and not (not df_temp.iloc[0, 5] and (df_temp.iloc[0, 2] - df_temp.iloc[0, 0]) >= 15):
            # Jump Command
            if not prevJump:
                UDPServerSocket.sendto(bytes.fromhex(
                    '06 01'), (localIP, gamePort))
                prevJump = True
            else:
                prevJump = False
        else:
            prevJump = False
        if df_temp.iloc[0, 6]:
            df2.columns = ['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance',
                           'Bar 2 Speed', 'Jump', 'Grounded', 'Dead', 'Scored']
            try:
                if exists("data.csv"):
                    pd.concat([pd.read_csv("data.csv"),
                           df2]).to_csv("data.csv", index=False)
                else:
                    df2.to_csv("data.csv", index=False)

                score = df2['Scored'].filter(lambda x: x == True).count()
                if exists("score.csv"):
                    pd.concat([pd.read_csv("score.csv"), pd.DataFrame(
                    [score], columns=['Score']).to_csv("score.csv", index=False)])
                else:
                    pd.DataFrame(
                    [score], columns=['Score']).to_csv("score.csv", index=False)
            except:
                pass
            
            df2 = pd.DataFrame()
            UDPServerSocket.sendto(bytes.fromhex('07 00'), (localIP, gamePort))

        df2 = pd.concat([df2, df_temp])
    except Exception as e:
        print(e)
        break


In [ ]:
# Export Data to CSV
df2.columns = ['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance',
               'Bar 2 Speed', 'Jump', 'Grounded', 'Dead', 'Scored']
try:
    pd.concat([pd.read_csv("data.csv"),
               df2]).to_csv("data.csv", index=False)
except:
    df2.to_csv("data.csv", index=False)


In [ ]:
# Close UDP Port When Game is Closed
UDPServerSocket.close()